##Introduzione  
Le reti siamesi sono reti neurali che condividono pesi tra due o più reti sorelle, ciascuna producendo vettori di incorporamento dei rispettivi input. Nell'apprendimento supervisionato per similarità, le reti vengono quindi addestrate per massimizzare il contrasto (distanza) tra incorporamenti di input di classi diverse, riducendo al minimo la distanza tra incorporamenti di classi simili, risultando in spazi di incorporamento che riflettono la segmentazione di classe degli input di formazione.

##Setup

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from tensorflow.python.ops.gen_array_ops import concat_v2_eager_fallback
from imutils import build_montages
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

from google.colab import drive
drive.mount('/content/drive')

epochs = 100
margin = 1 #Margine per la perdità di contrasto.

##Creare i dataset

In [ ]:
no_mask_train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/drive/Shareddrives/GRUPPI_CIMMINO/Aliberti/Datasets/NoMaskDataset',
  validation_split=0.5,
  subset="training",
  seed=123,
  image_size=(130, 130),
  batch_size=215
  )

no_mask_val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/drive/Shareddrives/GRUPPI_CIMMINO/Aliberti/Datasets/NoMaskDataset',
  validation_split=0.5,
  subset="validation",
  seed=123,
  image_size=(130, 130),
  batch_size=215
  )

mask_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/drive/Shareddrives/GRUPPI_CIMMINO/Aliberti/Datasets/MaskDataset',
  validation_split=0.5,
  subset="training",
  seed=123,
  image_size=(130, 130),
  batch_size= 215
  )

##Creare coppie di immagini  
Formeremo il modello per differenziare tra immagini di soggetti diversi. Per il momento utilizzeremo solo immagini di soggetti senza mascherina. Ad esempio, l'immagine appartenente al soggetto 1 deve essere differenziata dal resto delle immagini di altri soggetti (da 2 a 43), l'immagine del soggetto 2 differente dal soggetto 1 e da 3 a 43 e così via. Per fare ciò, selezioneremo N immagini casuali dalla classe A (ad esempio, per il soggetto 1) e le abbineremo a N immagini casuali di un'altra classe B (ad esempio, per il soggetto 2). Quindi, possiamo ripetere questo processo per tutte le classi di soggetti (fino al soggetto 43). Dopo aver accoppiato il soggetto 1 con altri soggetti, possiamo ripetere questo processo per le classi rimanenti per il resto dei soggetti (da 2 a 43).

In [ ]:
def make_pairs(images, labels):
	# initialize two empty lists to hold the (image, image) pairs and labels to indicate if a pair is positive or negative
	pairImages = []
	pairLabels = []
  # calculate the total number of classes present in the dataset and then build a list of indexes for each class label that provides the indexes for all examples with a given label
	numClasses = len(np.unique(labels))
	idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
  # loop over all images
	for idxA in range(len(images)):
		# grab the current image and label belonging to the current iteration
		currentImage = images[idxA]
		label = labels[idxA]
		# randomly pick an image that belongs to the *same* class label
		idxB = np.random.choice(idx[label])
		posImage = np.array(images)[idxB]
		# prepare a positive pair and update the images and labels lists, respectively
		pairImages.append([currentImage, posImage])
		pairLabels.append(1)
  	# grab the indices for each of the class labels *not* equal to the current label and randomly pick an image corresponding to a label *not* equal to the current label
		negIdx = np.where(labels != label)[0]
		negImage = images[np.random.choice(negIdx)]
		# prepare a negative pair of images and update our lists
		pairImages.append([currentImage, negImage])
		pairLabels.append(0)
	# return a 2-tuple of our image pairs and labels
	return (np.array(pairImages), np.array(pairLabels))
 
# load dataset
print("[INFO] Loading dataset...")
(trainX, trainY), (testX, testY) = no_mask_train_ds
trainX = trainX.astype("float32")
testX = testX.astype("float32")
(valX, valY), (testValX, testValY) = no_mask_val_ds
valX = valX.astype("float32")

# build the positive and negative image pairs
print("[INFO] Preparing positive and negative pairs...")

(pairTrain, labelTrain) = make_pairs(trainX, trainY)
print("[INFO] Completed Train...")

(pairVal, labelVal) = make_pairs(valX, valY)
print("[INFO] Completed Validation...")

(pairTest, labelTest) = make_pairs(testX, testY)
print("[INFO] Completed Test...")

##Dividere le coppie di training, le coppie di validation e le coppie di test

In [ ]:
x_train_1 = pairTrain[:, 0]  # x_train_1.shape is (430, 130, 130)
x_train_2 = pairTrain[:, 1]

x_val_1 = pairVal[:, 0]  # x_val_1.shape = (430, 130, 130)
x_val_2 = pairVal[:, 1]

x_test_1 = pairTest[:, 0]  # x_test_1.shape = (430, 130, 130)
x_test_2 = pairTest[:, 1]

##Visualizzare le coppie e le loro etichette

In [ ]:
def visualize(pairs, labels, to_show=6, num_col=3, predictions=None, test=False):
    """Creates a plot of pairs and labels, and prediction if it's test dataset.

    Arguments:
        pairs: Numpy Array, of pairs to visualize, having shape
               (Number of pairs, 2, 130, 130).
        to_show: Int, number of examples to visualize (default is 6)
                `to_show` must be an integral multiple of `num_col`.
                 Otherwise it will be trimmed if it is greater than num_col,
                 and incremented if if it is less then num_col.
        num_col: Int, number of images in one row - (default is 3)
                 For test and train respectively, it should not exceed 3 and 7.
        predictions: Numpy Array of predictions with shape (to_show, 1) -
                     (default is None)
                     Must be passed when test=True.
        test: Boolean telling whether the dataset being visualized is
              train dataset or test dataset - (default False).

    Returns:
        None.
    """

    # Define num_row
    # If to_show % num_col != 0
    #    trim to_show,
    #       to trim to_show limit num_row to the point where
    #       to_show % num_col == 0
    #
    # If to_show//num_col == 0
    #    then it means num_col is greater then to_show
    #    increment to_show
    #       to increment to_show set num_row to 1
    num_row = to_show // num_col if to_show // num_col != 0 else 1

    # `to_show` must be an integral multiple of `num_col`
    #  we found num_row and we have num_col
    #  to increment or decrement to_show
    #  to make it integral multiple of `num_col`
    #  simply set it equal to num_row * num_col
    to_show = num_row * num_col

    # Plot the images
    fig, axes = plt.subplots(num_row, num_col, figsize=(5, 5))
    for i in range(to_show):

        # If the number of rows is 1, the axes array is one-dimensional
        if num_row == 1:
            ax = axes[i % num_col]
        else:
            ax = axes[i // num_col, i % num_col]

        ax.imshow(tf.concat([pairs[i][0], pairs[i][1]], axis=1), cmap="gray")
        ax.set_axis_off()
        if test:
            ax.set_title("True: {} | Pred: {:.5f}".format(labels[i], predictions[i][0]))
        else:
            ax.set_title("Label: {}".format(labels[i]))
    if test:
        plt.tight_layout(rect=(0, 0, 5, 5), w_pad=0.0)
    else:
        plt.tight_layout(rect=(0, 0, 5, 5))
    plt.show()

#Ispezionare le coppie di training

In [ ]:
visualize(pairTrain.astype('uint8'), labelTrain.astype('uint8'), to_show=3, num_col=3)

##Ispezionare le coppie di validation

In [ ]:
visualize(pairVal.astype('uint8'), labelVal.astype('uint8'), to_show=3, num_col=3)

##Ispezionare le coppie di test

In [ ]:
visualize(pairTest.astype('uint8'), labelTest.astype('uint8'), to_show=3, num_col=3)

##Definire il modello  
Ci sono due livelli di input, ciascuno che porta alla propria rete, che produce incorporamenti. Un livello Lambda li unisce quindi utilizzando una distanza euclidea e l'output unito viene inviato alla rete finale.

In [ ]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))


input = layers.Input((130, 130, 3))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(4, (5, 5), activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, (5, 5), activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(10, activation="tanh")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((130, 130, 3))
input_2 = layers.Input((130, 130, 3))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

##Definire la perdita di contrasto


In [ ]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.

      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

##Compilare il modello con la perdita di contrasto

In [ ]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

##Addestrare il modello

In [ ]:
history = siamese.fit(
    [x_train_1, x_train_2],
    labelTrain,
    validation_data=([x_test_1, x_test_2], labelVal),
    #batch_size=64,
    epochs=10
)

##Visualizzare i risultati

In [ ]:
def plt_metric(history, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(history=history.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history.history, metric="loss", title="Constrastive Loss")

##Valutare il modello

In [ ]:
results = siamese.evaluate([x_test_1, x_test_2], labelTest)
print("test loss, test acc:", results)

##Visualizzare le previsioni

In [ ]:
predictions = siamese.predict([x_test_1, x_test_2])
visualize(pairTest.astype('uint8'), labelTest.astype('uint8'), to_show=4, predictions=predictions, test=True)

##Creare coppie finali di immagini
Formeremo il modello per differenziare immagini di stessi soggetti, ma con e senza mascherina. Ad esempio, l'immagine appartenente al soggetto 1, senza mascherina, deve essere differenziata dall'immagine del soggetto 1, con mascherina, l'immagine del soggetto 2, senza mascherina, differente dal soggetto 2, con mascherina, e così via. Per fare ciò, selezioneremo 1 immagine dalla classe A (ad esempio, per il soggetto 1 senza mascherina) e l'abbiniamo a una immagine di un'altra classe B (ad esempio, per il soggetto 1 con mascherina). Quindi, possiamo ripetere questo processo per tutte le classi di soggetti (fino al soggetto 43). Dopo aver accoppiato il soggetto 1 senza mascherina, con il soggetto 1 con mascherina, possiamo ripetere questo processo per le classi rimanenti per il resto dei soggetti (da 2 a 43).

In [ ]:
def make_final_pairs(images, labels):
	# initialize two empty lists to hold the (image, image) pairs and labels to indicate the number of class
	pairImages = []
	pairLabels = []
  # calculate the total number of classes present in the dataset
	numClasses = len(np.unique(labels))
	# loop for the total number of classes
	for idxA in range(numClasses):
		# loop over all images
		for idxC in range(len(images)):
			# grab the current image and label belonging to the current iteration
			if labels[idxC] == idxA:
				currentImage = images[idxC]
				label = labels[idxC]
				# loop over all images following the current image
				for idxB in range(idxC + 1, len(images)):
					# pick an image that belongs to the * same * subject class label with / without mask 
					if labels[idxB] == label:
						posImage = np.array(images)[idxB]
				break
		# prepare pair and update the images and labels lists, respectively
		pairImages.append([currentImage, posImage])
		pairLabels.append(label)
	# return a 2-tuple of our image pairs and labels
	return (np.array(pairImages), np.array(pairLabels))
 
# load dataset
print("[INFO] Loading dataset...")
(trainA, trainB), (testA, testB) = mask_ds
testA = testA.astype("float32")

resultX = tf.concat(axis=0, values = [testA, testX])
resultY = tf.concat(axis=0, values = [testB, testY])

# build the image pairs
print("[INFO] Preparing pairs...")
(pairResult, labelResult) = make_final_pairs(resultX, resultY)
print("[INFO] Completed result...")

##Ispezionare le coppie finali di result

In [ ]:
visualize(pairResult.astype('uint8'), labelResult.astype('uint8'), to_show=2, num_col=3)

##Dividere le coppie di result

In [ ]:
x_result_1 = pairResult[:, 0]  # x_result_1.shape is (43, 130, 130, 3)
x_result_2 = pairResult[:, 1]

##Visualizzare le previsioni di result

In [ ]:
predictions = siamese.predict([x_result_1, x_result_2])
visualize(pairResult.astype('uint8'), labelResult.astype('uint8'), to_show=43, num_col=4, predictions=predictions, test=True)